In [1]:
#convert excel to all be of same data type
import pandas
import numpy
import math
import random

#can probably make this a def function
pathname = "/Users/chugh/Documents/UFC Fight Code/Fighter stats train difference.csv"
pathname2 = "/Users/chugh/Documents/UFC Fight Code/Fighter stats train difference 2.csv"
stats = pandas.read_csv(pathname, index_col = 0)
stats = stats.reset_index()
#normalize columns
stats=(stats-stats.min())/(stats.max()-stats.min())
#print(stats)
stats2 = stats.astype('float32')
stats2.to_csv(pathname2, index=False)
stats3 = pandas.read_csv(pathname2, index_col = 0)
print(stats3)

            StrDef      TDpM     TDDef      Rank        WL  FinUFC       Age  \
SSpM                                                                           
0.781743  0.483333  0.000000  0.699387  0.675090  0.444444  0.2500  0.478261   
0.360166  0.550000  0.542923  0.141104  0.646209  0.444444  0.3750  0.652174   
0.658921  0.500000  0.394432  0.288344  0.671480  0.444444  0.5000  0.869565   
0.623237  0.566667  0.443929  0.343558  0.602888  0.000000  0.8125  0.826087   
0.854772  0.433333  0.555298  0.417178  0.707581  0.444444  0.5000  0.173913   
0.675519  0.533333  0.421500  0.552147  0.761733  0.555556  0.3750  0.391304   
0.858091  0.450000  0.215004  0.625767  0.700361  0.333333  0.7500  0.739130   
0.705394  0.583333  0.422274  0.539877  0.664260  0.555556  0.5625  0.347826   
0.721162  0.500000  0.482599  0.552147  0.638989  0.444444  0.5625  0.260870   
0.576763  0.533333  0.647332  0.809816  0.559567  0.333333  1.0000  0.608696   
0.566805  0.416667  0.440062  0.134969  

In [2]:
#want to write code that takes in two fighters stats and makes a prediction of who will win 
#additionally, would like to at some point make it so that it knows how they will win

#in order to do this, we will feed in data from both fighters as one data structure
#based on the features for each fighter, the algorithm will try to predict who will win

#we are going to want to create a structure of features and labels
#for features, we will include all data for Fighter 1 and Fighter 2 labeled F1 amd F2
#labels will contain the winner of each match (F1 or F2)

import os
import matplotlib.pyplot as plt
import tensorflow as tf
tf.enable_eager_execution()


#load in training data
pathname = "/Users/chugh/Documents/UFC Fight Code/Fighter stats train difference 2.csv"

# column order in CSV file
column_names = ['SSpM','StrDef','TDpM','TDDef','Rank','WL','FinUFC','Age','Dur','Result']


feature_names = column_names[:-1]
label_name = column_names[-1]

print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))

class_names = ['Fighter 1', 'Fighter 2']

batch_size = 5

train_dataset = tf.data.experimental.make_csv_dataset(
    pathname,
    batch_size,
    column_names=column_names,
    label_name=label_name,
    num_epochs=1)

features, labels = next(iter(train_dataset))
print(features)

Features: ['SSpM', 'StrDef', 'TDpM', 'TDDef', 'Rank', 'WL', 'FinUFC', 'Age', 'Dur']
Label: Result
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
OrderedDict([('SSpM', <tf.Tensor: id=86, shape=(5,), dtype=float32, numpy=
array([0.83651453, 0.69045645, 0.7427386 , 0.7294606 , 0.46639004],
      dtype=float32)>), ('StrDef', <tf.Tensor: id=87, shape=(5,), dtype=float32, numpy=
array([0.31666666, 0.55      , 0.43333334, 0.65      , 0.43333334],
      dtype=float32)>), ('TDpM', <tf.Tensor: id=89, shape=(5,), dtype=float32, numpy=
array([0.5011601 , 0.29156998, 0.38515082, 0.43078113, 0.3704563 ],
      dtype=float32)>), ('TDDef', <tf.Tensor: id=88, shape=(5,), dtype=float32, numpy=
array([0.19018404, 0.66871166, 0.34969324, 0.41104296, 0.41104296],
      dtype=float32)>), ('Rank', <tf.Tensor: id=85, shap

In [3]:
def pack_features_vector(features, labels):

    features = tf.stack(list(features.values()), axis=1)

    return features, labels
train_dataset = train_dataset.map(pack_features_vector)

features, labels = next(iter(train_dataset))

print(features[:5])

tf.Tensor(
[[0.83651453 0.31666666 0.5011601  0.19018404 0.833935   0.8888889
  0.         0.39130434 0.2742857 ]
 [0.69045645 0.55       0.29156998 0.66871166 0.58844763 0.7777778
  0.3125     0.3043478  0.3809524 ]
 [0.7427386  0.43333334 0.38515082 0.34969324 0.64620936 0.44444445
  0.875      0.5217391  0.42857143]
 [0.7294606  0.65       0.43078113 0.41104296 0.66064984 0.44444445
  0.625      0.5217391  0.23809524]
 [0.46639004 0.43333334 0.3704563  0.41104296 0.68231046 0.11111111
  0.75       0.65217394 0.08571429]], shape=(5, 9), dtype=float32)


In [4]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(20, activation=tf.nn.relu, input_shape=(9,)),  # input shape required
  tf.keras.layers.Dense(5, activation=tf.nn.relu),
  tf.keras.layers.Dense(2)
])

In [5]:
predictions = model(features)
tf.nn.softmax(predictions[:5])

<tf.Tensor: id=230, shape=(5, 2), dtype=float32, numpy=
array([[0.44972807, 0.550272  ],
       [0.48645332, 0.51354665],
       [0.48439217, 0.51560783],
       [0.47466132, 0.52533865],
       [0.47225016, 0.52774984]], dtype=float32)>

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    y_ = model(x, training=training)
    return loss_object(y_true=y, y_pred=y_)


l = loss(model, features, labels, training=False)
print("Loss test: {}".format(l))

Loss test: 0.641802191734314


In [7]:
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, training=True)

    return loss_value, tape.gradient(loss_value, model.trainable_variables)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
loss_value, grads = grad(model, features, labels)
print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),loss_value.numpy()))

optimizer.apply_gradients(zip(grads, model.trainable_variables))

print("Step: {}, Loss: {}".format(optimizer.iterations.numpy(),loss(model, features, labels, training=True).numpy()))

Step: 0, Initial Loss: 0.641802191734314
Step: 1, Loss: 0.6328970193862915


In [8]:
## Note: Rerunning this cell uses the same model variables

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    # Training loop - using batches of 32
    for x, y in train_dataset:
        # Optimize the model
        loss_value, grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Track progress
        epoch_loss_avg.update_state(loss_value)  # Add current batch loss
        # Compare predicted label to actual label
        # training=True is needed only if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        epoch_accuracy.update_state(y, model(x, training=True))

      # End epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())
    if epoch % 50 == 0:
        print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,epoch_loss_avg.result(),epoch_accuracy.result()))

Epoch 000: Loss: 0.702, Accuracy: 48.000%
Epoch 050: Loss: 0.690, Accuracy: 54.000%
Epoch 100: Loss: 0.686, Accuracy: 56.000%
Epoch 150: Loss: 0.683, Accuracy: 60.000%
Epoch 200: Loss: 0.678, Accuracy: 62.000%


In [9]:
import pandas
import numpy
import math

pathname = "/Users/chugh/Documents/UFC Fight Code/Fighter stats test difference.csv"
pathname2 = "/Users/chugh/Documents/UFC Fight Code/Fighter stats test difference 2.csv"
stats = pandas.read_csv(pathname, index_col = 0)
stats = stats.reset_index()
#normalize columns
stats=(stats-stats.min())/(stats.max()-stats.min())
stats2 = stats.astype('float32')
stats2.to_csv(pathname2, index=False)
stats3 = pandas.read_csv(pathname2, index_col = 0)


In [10]:
test_url = "/Users/chugh/Documents/UFC Fight Code/Fighter stats test difference 2.csv"

batch_size = 4
test_dataset = tf.data.experimental.make_csv_dataset(
    test_url,
    batch_size,
    column_names=column_names,
    label_name='Result',
    num_epochs=1,
    shuffle=False)

test_dataset = test_dataset.map(pack_features_vector)

In [11]:
test_accuracy = tf.keras.metrics.Accuracy()

for (x, y) in test_dataset:
  # training=False is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    logits = model(x, training=False)
    prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
    print(prediction)
    test_accuracy(prediction, y)

print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

tf.Tensor([0 0 0 0], shape=(4,), dtype=int32)
tf.Tensor([0 0 0 0], shape=(4,), dtype=int32)
tf.Tensor([0 1 0 0], shape=(4,), dtype=int32)
tf.Tensor([0 0 0], shape=(3,), dtype=int32)
Test set accuracy: 60.000%


In [12]:
#for new data
import pandas
import numpy
import math

pathname = "/Users/chugh/Documents/UFC Fight Code/Fighter Stats 81920 difference.csv"
pathname2 = "/Users/chugh/Documents/UFC Fight Code/Fighter Stats 81920 difference 2.csv"
stats = pandas.read_csv(pathname, index_col = 0)
stats = stats.reset_index()
#normalize columns
stats=(stats-stats.min())/(stats.max()-stats.min())
stats2 = stats.astype('float32')
stats2.to_csv(pathname2, index=False)

#will default results to 1
test_url = "/Users/chugh/Documents/UFC Fight Code/Fighter Stats 81920 difference 2.csv"

batch_size = 7 #batch size should be equal to the number of fights
test_dataset = tf.data.experimental.make_csv_dataset(
    test_url,
    batch_size,
    column_names=column_names,
    label_name='Result',
    num_epochs=1,
    shuffle=False)

test_dataset = test_dataset.map(pack_features_vector)

test_accuracy = tf.keras.metrics.Accuracy()

for (x, y) in test_dataset:
  # training=False is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    logits = model(x, training=False)
    prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
    print(prediction)
    test_accuracy(prediction, y)
    
tf.nn.softmax(logits)

tf.Tensor([1 1 1 1 1 0 0], shape=(7,), dtype=int32)


<tf.Tensor: id=222110, shape=(7, 2), dtype=float32, numpy=
array([[0.48082152, 0.51917845],
       [0.48626855, 0.5137314 ],
       [0.4853932 , 0.51460683],
       [0.47542468, 0.52457535],
       [0.43487683, 0.56512314],
       [0.5369226 , 0.4630774 ],
       [0.5369226 , 0.4630774 ]], dtype=float32)>

In [ ]:
model.save("/Users/chugh/Documents/UFC Fight Code/model_8320_hundred")

In [ ]:
print(column_names)